In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import time
from torchvision import datasets
import os

# Unzipping Files

## Instruction: Upload "archive(3).zip, Name new folder and title it "AI Images"


In [ ]:
!unzip "archive(3)".zip

# Data Prep

In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [7]:
data_dir = "Images"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform)
                  for x in ['train', 'test']}
print(len(image_datasets['train']))
print(len(image_datasets['test']))

100000
20000


## Choosing epochs, batch size, and learning rate

## Creating DataLoader

In [23]:
epochs = 30
batch = 32
learning_rate = 0.01

In [24]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes
print(class_names)

['FAKE', 'REAL']


# Creating ConvNet Class

In [11]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

# Prepping for Training

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training

In [27]:
start_time = time.time()
n_total_steps = len(dataloaders['train'])
num_epochs = epochs
start_time = time.time()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloaders['train']):

        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 1500 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print("--- %s seconds ---" % (time.time() - start_time))

Epoch [1/30], Step [1500/3125], Loss: 0.5714
Epoch [1/30], Step [3000/3125], Loss: 0.4600
Epoch [2/30], Step [1500/3125], Loss: 0.2671
Epoch [2/30], Step [3000/3125], Loss: 0.3171
Epoch [3/30], Step [1500/3125], Loss: 0.4445
Epoch [3/30], Step [3000/3125], Loss: 0.2564
Epoch [4/30], Step [1500/3125], Loss: 0.1635
Epoch [4/30], Step [3000/3125], Loss: 0.2604
Epoch [5/30], Step [1500/3125], Loss: 0.2537
Epoch [5/30], Step [3000/3125], Loss: 0.1394
Epoch [6/30], Step [1500/3125], Loss: 0.3585
Epoch [6/30], Step [3000/3125], Loss: 0.2353
Epoch [7/30], Step [1500/3125], Loss: 0.1823
Epoch [7/30], Step [3000/3125], Loss: 0.1779
Epoch [8/30], Step [1500/3125], Loss: 0.2519
Epoch [8/30], Step [3000/3125], Loss: 0.0797
Epoch [9/30], Step [1500/3125], Loss: 0.1317
Epoch [9/30], Step [3000/3125], Loss: 0.0768
Epoch [10/30], Step [1500/3125], Loss: 0.1030
Epoch [10/30], Step [3000/3125], Loss: 0.1528
Epoch [11/30], Step [1500/3125], Loss: 0.3859
Epoch [11/30], Step [3000/3125], Loss: 0.1023
Epoch 

# Saving Model

## DOWNLOAD .pth FILE BEFORE FINISHING

In [29]:
PATH = './e30_b32_lr01.pth'
torch.save(model.state_dict(), PATH)

## Loading Model on GPU (if needed)

In [ ]:
loaded_model = ConvNet().to(device)
loaded_model.load_state_dict(torch.load("cnn.pth"))
model = loaded_model

# Evaluating and Testing Model

In [28]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in dataloaders['test']:

        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the {len(dataloaders["test"])} test images: {acc} %')

Accuracy of the network on the 625 test images: 92.67 %
